In [21]:
import pandas as pd

url = 'https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv'
df = pd.read_csv(url)

df.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [22]:
df.shape

(891, 3)

In [23]:
df.duplicated().sum()

np.int64(0)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


Elimino la variable del nombre de la aplicación ya que lo unico que indica si un comentario es positivo o negativo es el propio comentario

In [25]:
df = df.drop(['package_name'], axis=1)
df.head()

,review,polarity
0,privacy at least put some option appear offli...,0
1,"messenger issues ever since the last update, ...",0
2,profile any time my wife or anybody has more ...,0
3,the new features suck for those of us who don...,0
4,forced reload on uploading pic on replying co...,0


In [27]:
df["review"] = df["review"].str.strip().str.lower()
df.head()

,review,polarity
0,privacy at least put some option appear offlin...,0
1,"messenger issues ever since the last update, i...",0
2,profile any time my wife or anybody has more t...,0
3,the new features suck for those of us who don'...,0
4,forced reload on uploading pic on replying com...,0


In [28]:
from sklearn.model_selection import train_test_split

X = df.drop("polarity", axis = 1)['review']
y = df["polarity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train.head()

331    just did the latest update on viber and yet ag...
733    keeps crashing it only works well in extreme d...
382    the fail boat has arrived the 6.0 version is t...
704    superfast, just as i remember it ! opera mini ...
813    installed and immediately deleted this crap i ...
Name: review, dtype: object

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

vec_model = CountVectorizer(stop_words = "english")
X_train = vec_model.fit_transform(X_train).toarray()
X_test = vec_model.transform(X_test).toarray()

Una vez convertido el texto en variables numéricas se puede entrenar el modelo, no tiene mayor sentido realizar EDA teniendo una sola variable como esta. Dado que tenemos datos discretos y me supongo que siguen una distribución multinomial, aplicaré 'MultinomialNB'

In [40]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [41]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
accuracy_score(y_pred, y_test)

0.8156424581005587

Voy a probar los otros modelos para ver si mi analisis fue correcto

In [38]:
from sklearn.naive_bayes import GaussianNB

model_Gaus = GaussianNB()
model_Gaus.fit(X_train, y_train)
y_pred_G = model_Gaus.predict(X_test)
accuracy_score(y_pred_G, y_test)

0.8044692737430168

In [39]:
from sklearn.naive_bayes import BernoulliNB

model_Berni = BernoulliNB()
model_Berni.fit(X_train, y_train)
y_pred_B = model_Berni.predict(X_test)
accuracy_score(y_pred_B, y_test)

0.770949720670391

Parece que mi analisis era correcto, aunque esperaba mayor diferencia entre la opción correcta y las incorrectas

Por último voy a intentar optimizar los hiperparametros del modelo bueno

In [63]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform  

param_distributions = {
    'alpha': uniform(loc=0.5, scale=10) ,
    'fit_prior' : [True, False]
}

model = MultinomialNB()

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=100,     
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Mejores parámetros encontrados:", random_search.best_params_)
print("Mejor accuracy durante validación:", random_search.best_score_)


Mejores parámetros encontrados: {'alpha': np.float64(1.8949386065204183), 'fit_prior': False}
Mejor accuracy durante validación: 0.8201910765291046


In [65]:
y_pred = random_search.predict(X_test)
print("Accuracy after optimization:", accuracy_score(y_test, y_pred))

Accuracy after optimization: 0.8212290502793296


In [75]:
model = MultinomialNB(alpha=6 , fit_prior=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8491620111731844

Como de costumbre probar cifras es da mejores resultados que la optimización, en cualquier caso el modelo ha mejorado hasta un más que decente 84.9% de precisión